In [10]:
import pandas as pd
import xml.etree.ElementTree as ET
import vertica_python
import re

In [11]:
def get_child_elements(root):
    # Create a list to store all child elements
    elements = []
    # Get all child elements of a given xml element
    for child in root:
        child.tag = re.sub(r"{.*?}", "", child.tag)
        elements.append(child)
        elements.extend(get_child_elements(child))
    return elements

def connect_to_database(db_config):
    # Create a connection to the Vertica database
    conn_info = {'host': db_config['host'],
                 'port': db_config['port'],
                 'user': db_config['user'],
                 'password': db_config['password'],
                 'database': db_config['database']}
    conn = vertica_python.connect(**conn_info)
    return conn

def execute_sql_query(cur, query, params=None):
    cur.execute(query, params)    
    return cur

def parse_xml_file(file_path, conn):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Get all child elements of the root element
    elements = get_child_elements(root)

    # Create a cursor
    cur = conn.cursor()

    # Iterate over the elements
    for element in elements:
        # Get the tag name
        tag_name = element.tag

        # Get the attributes of the element
        element_dict = element.attrib

        # Create a list of values to insert into the database
        values = tuple(element_dict.values())

        # Check if the table exists
        table_exists = execute_sql_query(cur, "SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = %s)", (tag_name,)).fetchone()[0]

        # If the table does not exist, create it
        if not table_exists:
            execute_sql_query(cur, f"CREATE TABLE {tag_name} ({','.join(f'{k} VARCHAR(255)' for k in element_dict.keys())})")

        # Create the insert statement
        insert_stmt = f"INSERT INTO {tag_name} VALUES ({','.join(['%s']*len(values))})"

        # Execute the insert statement
        execute_sql_query(cur, insert_stmt, values)

    # Commit the changes to the database
    conn.commit()



In [12]:

# Define file to read
#xml_file_path = r"C:\Users\Roberto\OneDrive\cargoIabono\Proyectos y Desarrollos\P001_V001_CFDI-Reader\01_Inputs\Nomina\EMS2103108P3_Pago de nómina_20220815_N_AOAR951019842.xml"
xml_file_path = r"C:\Users\Roberto\OneDrive\cargoIabono\Proyectos y Desarrollos\P001_V001_CFDI-Reader\01_Inputs\Facturas\0F23FE0D-8324-4BCE-AFAC-68CB67E89714.xml"
#xml_tree = ET.parse(xml_file_path)
#xml_root = xml_tree.getroot()

In [13]:
db_config = {'host': 'localhost',
             'port': 5433,
             'user': 'dbadmin',
             'password': '',
             'database': ''}

# Establish connection to the database
conn = connect_to_database(db_config)

# Parse and insert data from XML file into database
parse_xml_file(xml_file_path, conn)

# Close the connection
conn.close()


MissingSchema: Severity: ERROR, Message: Schema "information_schema" does not exist, Sqlstate: 3F000, Routine: RangeVarGetObjid, File: /data/jenkins/workspace/RE-ReleaseBuilds/RE-Knuckleboom/server/vertica/Catalog/Namespace.cpp, Line: 299, Error Code: 4650, SQL: "SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = 'Emisor')"